In [52]:
from dotenv import load_dotenv
from typing import TypedDict
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END, MessagesState

load_dotenv()

True

In [53]:
prompt = """
    You are a book expert.
    Your job is recommend books.
    Recommend some books in bases in the user request.
    If the user doesn't request an specific number of books, recomend 3.
    If the user request more than 5 books, recommend only 5 and tell the user that 
    you are only allow to recommend a maximun of 5 books.
    If the user doesn't ask for book recommendations or something similar, tell 
    the user that you are not capable of helping him.

    This is the user query:
    
"""

In [84]:
def nodo_inicio(state: MessagesState):
    print("Ejecutando nodo de inicio")
    
def nodo_intermedio(state: MessagesState):
    print("Ejecutando nodo intermedio")

    chain = ChatOpenAI(model="gpt-3.5-turbo")
    
    result = chain.invoke([SystemMessage(content=prompt), state["messages"][-1]])

    return {"messages": AIMessage(content=result.content)}

def nodo_final(state: MessagesState):
    print("Ejecutando nodo final")

In [85]:
builder = StateGraph(MessagesState)

builder.add_node("inicio", nodo_inicio)
builder.add_node("intermedio", nodo_intermedio)
builder.add_node("final", nodo_final)

builder.add_edge(START, "inicio")
builder.add_edge("inicio", "intermedio")
builder.add_edge("intermedio", "final")
builder.add_edge("final", END)


In [86]:
grafo_compilado = builder.compile()

In [87]:
mensaje_inicial = "Recommend me some books about individualism"

In [88]:
grafo_compilado.invoke({"messages": [HumanMessage(content=mensaje_inicial)]})

Ejecutando nodo de inicio
Ejecutando nodo intermedio
Ejecutando nodo final


{'messages': [HumanMessage(content='Recommend me some books about individualism', additional_kwargs={}, response_metadata={}, id='f92e385d-82e5-46c2-84e7-5244867a7b5b'),
  AIMessage(content='Sure! Here are 3 books about individualism that you might enjoy:\n\n1. "Atlas Shrugged" by Ayn Rand\n2. "Self-Reliance" by Ralph Waldo Emerson\n3. "The Fountainhead" by Ayn Rand\n\nIf you would like more recommendations, feel free to ask!', additional_kwargs={}, response_metadata={}, id='ece2bbfa-bce6-4b02-a2be-678095f530b8')]}

In [89]:
mensaje_erroneo = "Who is the current FC Barcelona manager?"

grafo_compilado.invoke({"messages": [HumanMessage(content=mensaje_erroneo)]})

Ejecutando nodo de inicio
Ejecutando nodo intermedio
Ejecutando nodo final


{'messages': [HumanMessage(content='Who is the current FC Barcelona manager?', additional_kwargs={}, response_metadata={}, id='f3b2c4c4-fa2d-4e71-a5ea-123c293c6a5d'),
  AIMessage(content="I'm sorry, I am not capable of helping with that question.", additional_kwargs={}, response_metadata={}, id='3f81ad27-6e22-4cfe-b766-3f87fd361a3f')]}